In [ ]:
import sys
sys.path.append("..")

import csaf
import csaf.utils as csafutils

from f16lib.acas import AcasScenario, AcasShieldScenario, AcasScenarioViewer
from f16lib.systems import F16AcasShieldIntruderBalloon

from IPython.display import HTML

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# check the system
AcasShieldScenario.system_type().check()

# view the system
csafutils.view_block_diagram(AcasShieldScenario.system_type, ipython_notebook=True)

In [ ]:
# construct a scenario
scen = AcasShieldScenario(
    [-1000, 5000], # balloon position
    500.0, # ownship airspeed
    ((-0.0, 4000.0, 1000.0),(-4000.0, 2500.0, 1000.0), (-8000, 4000, 1000.0)) # intruder waypoints
)

In [ ]:
sys = scen.create_system([6000, 8000, # relative position
                          -3.14/2, # relative heading
                          -200.0]) # relative airspeed
sys.check()

In [ ]:
trajs = sys.simulate_tspan((0.0, 30.0), show_status=True)

In [ ]:
viewer = AcasScenarioViewer(trajs, scen)
fig, ax = viewer.summary_plot()
plt.show()

In [ ]:
anim = viewer.summary_video()
HTML(anim.to_jshtml())